## **SetUp Colab**

In [ ]:
# Are we running in Colab
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

We're running Colab


In [ ]:
if IN_COLAB:
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)

  drive.mount(mount)

  # Switch to the directory on the Google Drive that you want to use
  import os
  drive_root = mount + "/My Drive/Colab Notebooks/Llama2-Translation/"

  # Create drive_root if it doesn't exist
  create_drive_root = True
  if create_drive_root:
    print("\nColab: making sure ", drive_root, " exists.")
    os.makedirs(drive_root, exist_ok=True)

  # Change to the directory
  print("\nColab: Changing directory to ", drive_root)
  %cd $drive_root

Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive

Colab: making sure  /content/gdrive/My Drive/Colab Notebooks/Llama2-Translation/  exists.

Colab: Changing directory to  /content/gdrive/My Drive/Colab Notebooks/Llama2-Translation/
/content/gdrive/My Drive/Colab Notebooks/Llama2-Translation


In [ ]:
# Verify we're in the correct working directory
%pwd

'/content/gdrive/My Drive/Colab Notebooks/Llama2-Translation'

In [ ]:
# All available files in the current working directory
!ls -a

base.txt		filtered_sentences_combined.csv  test_data_combined.txt
combined_sentences.txt	filtered_sentences_combined.txt  test.txt
converted.txt		.ipynb_checkpoints		 train_data_combined.txt
data			output.csv			 train.txt
data-mnt		output.txt			 validation_data_combined.txt
Data-Preprocess.ipynb	remove_punc_combined.txt	 validation.txt


In [ ]:
# Setting file explorer to view current directory only
from google.colab import files
files.view(drive_root)

<IPython.core.display.Javascript object>

## **Load HuggingFace Dataset**

In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("Helsinki-NLP/opus-100", "bn-en")

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
print(f"Train dataset size: {len(dataset['train'])}")

Train dataset size: 1000000


In [ ]:
dataset

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})

In [ ]:
dataset['train']['translation'][:5]

[{'bn': 'হ্যাঁ?', 'en': 'Yeah?'},
 {'bn': 'তিনি বললেনঃ অতঃপর তোমাদের প ্ রধান উদ ্ দেশ ্ য কি হে আল ্ লাহর প ্ রেরিতগণ ?',
  'en': 'Messengers , what is your task ? "'},
 {'bn': 'ই-মেইল অ্যাকাউন্টগুলিকে এখানে কনফিগার করুন',
  'en': 'Configure email accounts'},
 {'bn': 'আমি যখন মানুষের প ্ রতি অনুগ ্ রহ করি তখন সে মুখ ফিরিয়ে নেয় এবং পার ্ শ ্ ব পরিবর ্ তন করে । আর যখন তাকে অনিষ ্ ট স ্ পর ্ শ করে , তখন সুদীর ্ ঘ দোয়া করতে থাকে ।',
  'en': 'When We bestow Our favour upon man , he turns away and waxes proud ; but when a misfortune touches him , he is full of supplication .'},
 {'bn': '- ঠিক আছে.', 'en': '- Okay.'}]

In [ ]:
data_train = dataset['train']['translation']

In [ ]:
data_test = dataset['test']['translation']

In [ ]:
data_val = dataset['validation']['translation']

In [ ]:
# [
#     {
#      'bn': 'হ্যাঁ?',
#      'en': 'Yeah?'
#     },
#     {
#      'bn': 'তিনি বললেন',
#      'en': 'He say'
#     },
# ]

# হ্যাঁ? ###>Yeah?
# তিনি বললেন ###>He say

In [ ]:
with open('validation.txt', 'w', encoding='utf-8') as f:
    for item in data_test:
        f.write(f"{item['bn']} ###>{item['en']}\n")

In [ ]:
import csv

# Read lines from the text file
with open('validation.txt', 'r', encoding='utf-8') as text_file:
    lines = text_file.readlines()

# Write lines into a CSV file
with open('validation.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)

    # Write the column name
    writer.writerow(['translations'])

    # Write each line from the text file into the CSV
    for line in lines:
        writer.writerow([line.strip()])


In [ ]:
import pandas as pd

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('output.csv')

# Display the DataFrame
df.head()


In [ ]:
with open('base.txt', 'r', encoding='utf-8') as base_file:
    with open('converted.txt', 'w', encoding='utf-8') as converted_file:
        for line in base_file:
            converted_line = line.replace('||| ', '###>')
            converted_file.write(converted_line)


In [ ]:
dataset_bnmnt = load_dataset("csebuetnlp/BanglaNMT")

Generating train split:   0%|          | 0/2379749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/597 [00:00<?, ? examples/s]

In [ ]:
dataset_bnmnt

DatasetDict({
    train: Dataset({
        features: ['bn', 'en'],
        num_rows: 2379749
    })
    test: Dataset({
        features: ['bn', 'en'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['bn', 'en'],
        num_rows: 597
    })
})

In [ ]:
dataset_bnmnt['train'][:5]

{'bn': ['আমাকে সব কিছু জানাও',
  '"এ্যামোর এন্ড সাইকি", এটা খারাপ, মনিব এটায় প্রচুর পরিমাণে চিরহরিৎ (একপ্রকার সুগন্ধী লতা) আছে আর প্রচুর পরিমাণে...',
  'কাছানো',
  'পাকিস্তানের জাতীয় পতাকা',
  'দন্তোদ্গম'],
 'en': ['Just keep me informed.',
  'It\'s bad, "Amor and Psyche" is, master.',
  'Overtake',
  'Flag of Pakistan',
  'cutting of teeth']}

In [ ]:
dataset_bnmnt['train']['bn'][:5]

['আমাকে সব কিছু জানাও',
 '"এ্যামোর এন্ড সাইকি", এটা খারাপ, মনিব এটায় প্রচুর পরিমাণে চিরহরিৎ (একপ্রকার সুগন্ধী লতা) আছে আর প্রচুর পরিমাণে...',
 'কাছানো',
 'পাকিস্তানের জাতীয় পতাকা',
 'দন্তোদ্গম']

In [ ]:
dataset_bnmnt['train']['en'][:5]

['Just keep me informed.',
 'It\'s bad, "Amor and Psyche" is, master.',
 'Overtake',
 'Flag of Pakistan',
 'cutting of teeth']

In [ ]:
len(dataset_bnmnt['train']['bn'])

2379749

In [ ]:
len(dataset_bnmnt['train']['en'])

2379749

In [ ]:
dataset_bnmnt_train_bn = dataset_bnmnt['train']['bn']
dataset_bnmnt_train_en = dataset_bnmnt['train']['en']

In [ ]:
# Split each sentence into tokens and find their lengths
token_lengths = [len(sentence.split()) for sentence in dataset_bnmnt_train_bn]

# Find maximum and minimum token lengths
max_token_length = max(token_lengths)
min_token_length = min(token_lengths)

print("Maximum token length:", max_token_length)
print("Minimum token length:", min_token_length)

Maximum token length: 1069
Minimum token length: 1


In [ ]:
# Split each sentence into tokens and find their lengths
token_lengths = [len(sentence.split()) for sentence in dataset_bnmnt_train_en]

# Find maximum and minimum token lengths
max_token_length = max(token_lengths)
min_token_length = min(token_lengths)

print("Maximum token length:", max_token_length)
print("Minimum token length:", min_token_length)

Maximum token length: 1203
Minimum token length: 1


In [ ]:
# 10000 <= number <= 30000

In [ ]:
# Filter sentences with token lengths less than 100
filtered_sentences_bn = [sentence for sentence in dataset_bnmnt_train_bn if 2 <= len(sentence.split()) <= 25 ]

In [ ]:
len(filtered_sentences_bn)
# 2379749

1810783

In [ ]:
# Filter sentences with token lengths less than 100
filtered_sentences_en = [sentence for sentence in dataset_bnmnt_train_en if len(sentence.split()) < 25]

In [ ]:
len(filtered_sentences_en)
# 2379749

2063417

In [ ]:
# Combine sentences into a list with desired format
combined_list = [f"{bn} ###>{en}" for bn, en in zip(dataset_bnmnt_train_bn, dataset_bnmnt_train_en)]

# Write combined list to a text file
with open('combined_sentences.txt', 'w', encoding='utf-8') as file:
    for sentence in combined_list:
        file.write(sentence + '\n')

# Print the combined list
print(len(combined_list))

2379749


In [ ]:
combined_list[:5]

['আমাকে সব কিছু জানাও ###>Just keep me informed.',
 '"এ্যামোর এন্ড সাইকি", এটা খারাপ, মনিব এটায় প্রচুর পরিমাণে চিরহরিৎ (একপ্রকার সুগন্ধী লতা) আছে আর প্রচুর পরিমাণে... ###>It\'s bad, "Amor and Psyche" is, master.',
 'কাছানো ###>Overtake',
 'পাকিস্তানের জাতীয় পতাকা ###>Flag of Pakistan',
 'দন্তোদ্গম ###>cutting of teeth']

In [ ]:
def decontracting_words(sentence):
    """
    Decontracting words (e.g. I'm -> I am, I've -> I have, etc.)
    https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    https://stackoverflow.com/a/19794953
    """
    contractions = {
        "ain't": "am not",
        "aren't": "are not",
        "can't": "can not",
        "can't've": "can not have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he would",
        "he'd've": "he would have",
        "he'll": "he will",
        "he'll've": "he will have",
        "he's": "he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how is",
        "i'd": "i would",
        "i'd've": "i would have",
        "i'll": "i will",
        "i'll've": "i will have",
        "i'm": "i am",
        "i've": "i have",
        "isn't": "is not",
        "it'd": "it would",
        "it'd've": "it would have",
        "it'll": "it will",
        "it'll've": "it will have",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she would",
        "she'd've": "she would have",
        "she'll": "she will",
        "she'll've": "she will have",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so as",
        "that'd": "that would",
        "that'd've": "that would have",
        "that's": "that is",
        "there'd": "there would",
        "there'd've": "there would have",
        "there's": "there is",
        "they'd": "they would",
        "they'd've": "they would have",
        "they'll": "they will",
        "they'll've": "they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what will",
        "what'll've": "what will have",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "when's": "when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where is",
        "where've": "where have",
        "who'll": "who will",
        "who'll've": "who will have",
        "who's": "who is",
        "who've": "who have",
        "why's": "why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you would",
        "you'd've": "you would have",
        "you'll": "you will",
        "you'll've": "you will have",
        "you're": "you are",
        "you've": "you have"
    }

    sentence_decontracted = []

    for word in sentence.split():
        if word in contractions:
            word = contractions[word]

        sentence_decontracted.append(word)

    sentence = ' '.join(sentence_decontracted)
    sentence = sentence.replace("'ve", " have")
    sentence = sentence.replace("n't", " not")
    sentence = sentence.replace("'re", " are")
    sentence = sentence.replace("'ll", " will")
    sentence = sentence.replace("'d", " would")
    sentence = sentence.replace("'s", " is")
    sentence = sentence.replace("'m", " am")

    return sentence

In [ ]:
decon_combined_list = [decontracting_words(sentence) for sentence in combined_list]

In [ ]:
len(decon_combined_list)

2379749

In [ ]:
decon_combined_list[:5]

['আমাকে সব কিছু জানাও ###>Just keep me informed.',
 '"এ্যামোর এন্ড সাইকি", এটা খারাপ, মনিব এটায় প্রচুর পরিমাণে চিরহরিৎ (একপ্রকার সুগন্ধী লতা) আছে আর প্রচুর পরিমাণে... ###>It is bad, "Amor and Psyche" is, master.',
 'কাছানো ###>Overtake',
 'পাকিস্তানের জাতীয় পতাকা ###>Flag of Pakistan',
 'দন্তোদ্গম ###>cutting of teeth']

In [ ]:
import re

def remove_punc(sentence):
    # initializing punctuations string
    punc = """!()-[]{}।;:'"\,<./?@$%^&*_~"""

    for letter in sentence:
        if letter in punc:
            sentence = sentence.replace(letter, "")

    sentence = re.sub("\s{2,}", " ", sentence)

    return sentence

In [ ]:
remove_punc_combined_list = [remove_punc(sentence) for sentence in decon_combined_list]

In [ ]:
len(remove_punc_combined_list)

2379749

In [ ]:
remove_punc_combined_list[:5]

['আমাকে সব কিছু জানাও ###>Just keep me informed',
 'এ্যামোর এন্ড সাইকি এটা খারাপ মনিব এটায় প্রচুর পরিমাণে চিরহরিৎ একপ্রকার সুগন্ধী লতা আছে আর প্রচুর পরিমাণে ###>It is bad Amor and Psyche is master',
 'কাছানো ###>Overtake',
 'পাকিস্তানের জাতীয় পতাকা ###>Flag of Pakistan',
 'দন্তোদ্গম ###>cutting of teeth']

In [ ]:
# Split each sentence into tokens and find their lengths
token_lengths = [len(sentence.split()) for sentence in remove_punc_combined_list]

# Find maximum and minimum token lengths
max_token_length = max(token_lengths)
min_token_length = min(token_lengths)

print("Maximum token length:", max_token_length)
print("Minimum token length:", min_token_length)

Maximum token length: 2278
Minimum token length: 2


In [ ]:
# Filter sentences with token lengths less than 100
filtered_sentences_combined = [sentence for sentence in remove_punc_combined_list if 2 <= len(sentence.split()) <= 45 ]
len(filtered_sentences_combined)
# 2379749

2051053

In [ ]:
# Write filtered_sentences_combined list to a text file
with open('filtered_sentences_combined.txt', 'w', encoding='utf-8') as file:
    for sentence in filtered_sentences_combined:
        file.write(sentence + '\n')

In [ ]:
import csv

# Read lines from the text file
with open('filtered_sentences_combined.txt', 'r', encoding='utf-8') as text_file:
    lines = text_file.readlines()

# Write lines into a CSV file
with open('filtered_sentences_combined.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)

    # Write the column name
    writer.writerow(['translations'])

    # Write each line from the text file into the CSV
    for line in lines:
        writer.writerow([line.strip()])

In [ ]:
import math
math.ceil(2051053 * 0.8)

1640843

In [ ]:
math.floor(2051053 * 0.1)

205105

In [ ]:
205105*2

410210

In [ ]:
1640843+410210

2051053

In [ ]:
train_data_combined = filtered_sentences_combined[:1640843]
validation_data_combined = filtered_sentences_combined[1640843-1:1640843+205105]
test_data_combined = filtered_sentences_combined[1640843+205105-1:2051053]

In [ ]:
len(train_data_combined)

1640843

In [ ]:
len(validation_data_combined)

205106

In [17]:
len(test_data_combined)

NameError: name 'test_data_combined' is not defined

In [ ]:
# Write filtered_sentences_combined list to a text file
with open('test_data_combined.txt', 'w', encoding='utf-8') as file:
    for sentence in test_data_combined:
        file.write(sentence + '\n')

In [ ]:
import csv

# Read lines from the text file
with open('test_data_combined.txt', 'r', encoding='utf-8') as text_file:
    lines = text_file.readlines()

# Write lines into a CSV file
with open('test_data_combined.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)

    # Write the column name
    writer.writerow(['translations'])

    # Write each line from the text file into the CSV
    for line in lines:
        writer.writerow([line.strip()])

## New

In [ ]:
dataset_danish = load_dataset("kaitchup/opus-Danish-to-English")

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/946341 [00:00<?, ? examples/s]

In [ ]:
dataset_danish

DatasetDict({
    validation: Dataset({
        features: ['text'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['text'],
        num_rows: 946341
    })
})

In [ ]:
dataset_danish['train']

Dataset({
    features: ['text'],
    num_rows: 946341
})

In [ ]:
dataset_danish['train']['text'][:5]

['På Det Blandede EØS-Udvalgs vegne ###>For the EEA Joint Committee',
 'metal, der indeholder mindst 99,9 vægtprocent bly, forudsat ingen anden bestanddel indgår i mængder, der overstiger de i nedenstående skema anførte grænseværdier: ###>Metal containing by weight at least 99,9 % of lead, provided that the content by weight of any other element does not exceed the limit specified in the following table:',
 'Tænk. ###>Think.',
 'Beth... ###>Beth...',
 'Vort projekt "Menneskelig dvale" gør det muligt at holde vores bedste mænd nedfrosset i deres bedste tilstand, for at bruge dem efter behov. ###>With the Human Hibernation Project, we will be able to save our best men... frozen in their prime, for use when they are needed most.']

In [ ]:
dataset_bn = load_dataset("musfiqdehan/preprocessed-BanglaNMT")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset_bn

DatasetDict({
    train: Dataset({
        features: ['translations'],
        num_rows: 1640843
    })
    validation: Dataset({
        features: ['translations'],
        num_rows: 205106
    })
    test: Dataset({
        features: ['translations'],
        num_rows: 205106
    })
})

In [ ]:
dataset_bn['train'][]

Dataset({
    features: ['translations'],
    num_rows: 1640843
})

In [ ]:
dataset_bn['train']['translations'][:5]

['আমাকে সব কিছু জানাও ###>Just keep me informed',
 'এ্যামোর এন্ড সাইকি এটা খারাপ মনিব এটায় প্রচুর পরিমাণে চিরহরিৎ একপ্রকার সুগন্ধী লতা আছে আর প্রচুর পরিমাণে ###>It is bad Amor and Psyche is master',
 'কাছানো ###>Overtake',
 'পাকিস্তানের জাতীয় পতাকা ###>Flag of Pakistan',
 'দন্তোদ্গম ###>cutting of teeth']

## Small Dataset

In [19]:
%pwd

'/content/gdrive/MyDrive/Colab Notebooks/Llama2-Translation'

### Import Libraries

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

### Read The Text File

In [20]:
with open('test_data_combined.txt') as file:
    lines = file.readlines()

### Split The Lines

In [21]:
train_lines, temp_lines = train_test_split(lines, test_size=0.2, random_state=42)
val_lines, test_lines = train_test_split(temp_lines, test_size=0.5, random_state=42)

### Create DataFrames and add header

In [22]:
train_df = pd.DataFrame(train_lines)
train_df.columns = ['translations']

val_df = pd.DataFrame(val_lines)
val_df.columns = ['translations']

test_df = pd.DataFrame(test_lines)
test_df.columns = ['translations']

### Save as CSV Files

In [23]:
train_df.to_csv('small-dataset/train.csv', index=False)
val_df.to_csv('small-dataset/validation.csv', index=False)
test_df.to_csv('small-dataset/test.csv', index=False)

### Count No of Lines

In [24]:
import pandas as pd

# Read the CSV files
train_df = pd.read_csv('small-dataset/train.csv')
val_df = pd.read_csv('small-dataset/validation.csv')
test_df = pd.read_csv('small-dataset/test.csv')

# Count the number of lines in each file
train_lines = len(train_df.index)
val_lines = len(val_df.index)
test_lines = len(test_df.index)

# Print the results
print(f'Number of lines in train.csv: {train_lines}')
print(f'Number of lines in validation.csv: {val_lines}')
print(f'Number of lines in test.csv: {test_lines}')

Number of lines in train.csv: 164084
Number of lines in validation.csv: 20511
Number of lines in test.csv: 20511
